In [1]:
# Imports
import scipy.io
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load the .mat file
data = scipy.io.loadmat('data/condsForSimJ2moMuscles.mat')

# Extract condsForSim struct
conds_for_sim = data['condsForSim']

In [3]:
# Initialize lists to store data for all conditions
go_envelope_all = []
plan_all = []
muscle_all = []

# Get the number of conditions (rows) and delay durations (columns)
num_conditions, num_delays = conds_for_sim.shape

# Loop through each condition and extract data
for i in range(num_conditions):  # 27 conditions
    go_envelope_condition = []
    plan_condition = []
    muscle_condition = []

    for j in range(num_delays):  # 8 delay durations
        condition = conds_for_sim[i, j]

        go_envelope = condition['goEnvelope']
        plan = condition['plan']
        muscle = condition['muscle']

        # Select only muscles 5 and 6 
        selected_muscle_data = muscle[:, [4, 5]]  # Select columns for muscle 5 and 6, which show the nicest multiphasic activity

        go_envelope_condition.append(go_envelope)
        plan_condition.append(plan)
        muscle_condition.append(selected_muscle_data)

    # Stack data for each condition
    go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))
    plan_all.append(torch.tensor(plan_condition, dtype=torch.float32))
    muscle_all.append(torch.tensor(muscle_condition, dtype=torch.float32))

# Stack data for all conditions
go_envelope_tensor = torch.stack(go_envelope_all)
plan_tensor = torch.stack(plan_all)
muscle_tensor = torch.stack(muscle_all)

# Reshape to merge the first two dimensions
go_envelope_tensor = go_envelope_tensor.reshape(-1, *go_envelope_tensor.shape[2:])
plan_tensor = plan_tensor.reshape(-1, *plan_tensor.shape[2:])
muscle_tensor = muscle_tensor.reshape(-1, *muscle_tensor.shape[2:])

# Print shapes
print(f"Go Envelope Tensor Shape: {go_envelope_tensor.shape}")
print(f"Plan Tensor Shape: {plan_tensor.shape}")
print(f"Muscle Tensor Shape: {muscle_tensor.shape}")


/tmp/ipykernel_214548/1417502240.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  go_envelope_all.append(torch.tensor(go_envelope_condition, dtype=torch.float32))


Go Envelope Tensor Shape: torch.Size([216, 296, 1])
Plan Tensor Shape: torch.Size([216, 296, 15])
Muscle Tensor Shape: torch.Size([216, 296, 2])


In [4]:
# Normalize and standardize a tensor
def normalize_and_standardize(tensor):
    # Normalize: Scale to 0-1 range
    min_val = tensor.min()
    max_val = tensor.max()
    tensor = (tensor - min_val) / (max_val - min_val)

    # Standardize: Shift to zero mean and unit variance
    mean = tensor.mean()
    std = tensor.std()
    tensor = (tensor - mean) / std

    return tensor

# Apply the function to each tensor
normalized_go_envelope_tensor = normalize_and_standardize(go_envelope_tensor)
normalized_plan_tensor = normalize_and_standardize(plan_tensor)
normalized_muscle_tensor = normalize_and_standardize(muscle_tensor)

# Print shapes to confirm
print(f"Normalized Go Envelope Tensor Shape: {normalized_go_envelope_tensor.shape}")
print(f"Normalized Plan Tensor Shape: {normalized_plan_tensor.shape}")
print(f"Normalized Muscle Tensor Shape: {normalized_muscle_tensor.shape}")


Normalized Go Envelope Tensor Shape: torch.Size([216, 296, 1])
Normalized Plan Tensor Shape: torch.Size([216, 296, 15])
Normalized Muscle Tensor Shape: torch.Size([216, 296, 2])


In [5]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, g, tau=50):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau  # Time constant

        # Weight initialization
        self.J = nn.Parameter(torch.randn(hidden_size, hidden_size) * (g / torch.sqrt(torch.tensor(hidden_size, dtype=torch.float))))
        self.B = nn.Parameter(torch.randn(hidden_size, input_size) / torch.sqrt(torch.tensor(input_size, dtype=torch.float)))
        self.w = nn.Parameter(torch.zeros(output_size, hidden_size))
        self.bx = nn.Parameter(torch.zeros(hidden_size))
        self.bz = nn.Parameter(torch.zeros(output_size))

        # Nonlinearity
        self.nonlinearity = torch.tanh
    
    def forward(self, x, hidden):
        # x is of shape [batch_size, input_size]
        # hidden is of shape [1, hidden_size] or [batch_size, hidden_size]
    
        # Matrix multiplication
        hidden_update = torch.matmul(self.J, hidden.T)  # [hidden_size, batch_size]
        input_update = torch.matmul(self.B, x.T)  # [hidden_size, batch_size]
    
        # Apply nonlinearity
        new_hidden = self.nonlinearity(hidden_update + input_update + self.bx.unsqueeze(1))
    
        # Transpose back to [batch_size, hidden_size]
        new_hidden = new_hidden.T
    
        output = torch.matmul(self.w, new_hidden.T) + self.bz.unsqueeze(1)
        output = output.T  # Transpose to [batch_size, output_size]
    
        return output, new_hidden


    
    def init_hidden(self):
        # Initialize hidden state with an additional batch dimension
        return torch.zeros(1, self.hidden_size)


# Hyperparameters
input_size = 15
hidden_size = 300
output_size = 2  # Number of muscles
g = 1.1  # Moderate g value for simpler dynamics

# Model instantiation
model = SimpleRNN(input_size, hidden_size, output_size, g)

In [6]:
# Prepare data for training
# Assuming that 'normalized_plan_tensor' is your input data and 'normalized_muscle_tensor' is your target data
X_train = normalized_plan_tensor
y_train = normalized_muscle_tensor

In [7]:
# Convert datasets to TensorDataset and DataLoader for batch processing
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64  # You can adjust this based on your data size and memory constraints
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [8]:
# Loss Function and Optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training Loop
num_epochs = 500  # The number of times the entire dataset is passed through the network

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        h = model.init_hidden()

        # Process each time step
        for t in range(inputs.shape[1]):  # iterate over time steps
            output, h = model(inputs[:, t, :], h)

        # Compute loss using the last output (if your task is many-to-one)
        loss = criterion(output, targets[:, -1, :])
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')


Epoch 1, Loss: 0.46963347494602203
Epoch 2, Loss: 0.45373592525720596
Epoch 3, Loss: 0.5241474062204361
Epoch 4, Loss: 0.4954524487257004
Epoch 5, Loss: 0.46555476635694504
Epoch 6, Loss: 0.5544778257608414
Epoch 7, Loss: 0.475787989795208
Epoch 8, Loss: 0.4912019148468971
Epoch 9, Loss: 0.4985133558511734
Epoch 10, Loss: 0.4873269721865654
Epoch 11, Loss: 0.4678957387804985
Epoch 12, Loss: 0.465007059276104
Epoch 13, Loss: 0.46721724420785904
Epoch 14, Loss: 0.4783782735466957
Epoch 15, Loss: 0.4777790829539299
Epoch 16, Loss: 0.4955277368426323
Epoch 17, Loss: 0.5189576596021652
Epoch 18, Loss: 0.542348101735115
Epoch 19, Loss: 0.5266738310456276
Epoch 20, Loss: 0.5250164568424225
Epoch 21, Loss: 0.48532481491565704
Epoch 22, Loss: 0.49602771550416946
Epoch 23, Loss: 0.5364746451377869
Epoch 24, Loss: 0.5523156896233559
Epoch 25, Loss: 0.472917839884758
Epoch 26, Loss: 0.43356650322675705
Epoch 27, Loss: 0.4699414595961571
Epoch 28, Loss: 0.5136973261833191
Epoch 29, Loss: 0.45945645